In [1]:
import tensorflow as tf
import numpy as np
from keras.datasets import mnist
from datagen import train_generator, zca_whitening
from tqdm import tqdm
from scipy.misc import imresize

Using TensorFlow backend.


In [2]:
time_len=10
batch_size=32
resize=(32, 32, 3)

# prev, obs, actions = np.load(mode+'_frames.npy')
print("Loading...")
D = np.load('./animal_set.npy')
B = np.load('./coil_100.npy')
if resize is not None:
    print("Resizing...")
    shape = (batch_size, time_len) + resize
    newD = np.zeros(D.shape[:2] + resize)
    newB = np.zeros(B.shape[:2] + resize)
    for i in range(D.shape[0]):
        for j in range(D.shape[1]):
            newD[i, j] = imresize(D[i, j], resize)
    for i in range(B.shape[0]):
        for j in range(B.shape[1]):
            newB[i, j] = imresize(B[i, j], resize)
    D = newD
    B = newB
else:
    shape = (batch_size, time_len) + (128, 128, 3)
D = D/127.5 - 1
B = B/127.5 - 1
print('ZCA...')
D, ZCA, mean = zca_whitening(D)
oshape = (-1, 72, 32, 32, 3)
B = B.reshape(np.prod(B.shape[:2]), -1)
B = B - mean
B = np.dot(ZCA, B.T).T
B = B.reshape(oshape)
print(D.shape, D.min(), D.max())

Loading...
Resizing...
ZCA...
(3072, 3072)
((3072, 3072), (3072,))
(3072, 3072)
((60, 65, 32, 32, 3), -1.7847253545301025, 3.2941168584318432)


In [3]:
# from models import baseline as model
from models import rwta as model

In [4]:
name = 'rwta'
batch = 10
Out = np.zeros(B.shape[:4]+(128,))
with tf.Session() as sess:
    ftrain, ftest, loader, saver, extras = model.get_model(
        sess=sess, name=name, batch_size=batch, time_len=B.shape[1])
    loader()
    total_cost = 0.
    for i in tqdm(range(0, B.shape[0], batch)):
        o = extras[1](B[i:i+batch])
        Out[i:i+batch] = o.reshape(batch, 72, 32, 32, 128)

[<tensorflow.python.ops.variables.Variable object at 0x7f1ab068ca90>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab06f9090>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab06bc790>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab06b46d0>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0650210>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0661710>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0661fd0>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0605b10>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0605550>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab0619a50>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab062c9d0>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab063b910>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab05e49d0>, <tensorflow.python.ops.variables.Variable object at 0x7f1ab05cff10>, <tensorflow.python.ops.variables.

  0%|          | 0/10 [00:00<?, ?it/s]

 [*] Checkpoints read: rwta-0


100%|██████████| 10/10 [00:22<00:00,  2.34s/it]


In [5]:
Ytr = np.zeros((100, 4, 1))
Yts = np.zeros((100, 72-4, 1))
for i in range(100):
    Ytr[i, :, :] = i * np.ones((4, 1))
    Yts[i, :, :] = i * np.ones((72-4, 1))

Xtr = np.zeros((100, 4, 32, 32, 128))
Xts = np.zeros((100, 72-4, 32, 32, 128))
tridx = [0, 18, 36, 54]
tsidx = [i for i in range(72) if i not in tridx]


for i in range(100):
    Xtr[i, :] = Out[i, tridx]
    Xts[i, :] = Out[i, tsidx]
    
Xtr = Xtr.reshape(-1, 32*32*128)
Ytr = Ytr.reshape(-1, 1)
Xts = Xts.reshape(-1, 32*32*128)
Yts = Yts.reshape(-1, 1)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [7]:
clf = Pipeline([('svc', SVC(kernel='linear'))])

In [8]:
clf.fit(Xtr, Ytr)

/home/edersantana/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [9]:
clf.score(Xts, Yts)

0.85029411764705887